In [5]:
import os

# 👇 EXACT path to your project folder
os.chdir(r"C:\Users\bynigeri\OneDrive - UAB - The University of Alabama at Birmingham\Documents\Analyst Projects\Customer churn analytics")

print("Now PWD:", os.getcwd())
print("Here:", os.listdir())


Now PWD: C:\Users\bynigeri\OneDrive - UAB - The University of Alabama at Birmingham\Documents\Analyst Projects\Customer churn analytics
Here: ['.git', '.gitignore', 'data', 'docs', 'models', 'notebooks', 'README.md']


In [6]:
import pandas as pd
import numpy as np

data_path = "data/raw/telco_churn_raw.csv"  # 👈 no ../ anymore

df = pd.read_csv(data_path)

df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
df['Churn'].value_counts()


No     5174
Yes    1869
Name: Churn, dtype: int64

In [9]:
# Make column names easier to work with
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

df.columns.tolist()



['customerid',
 'gender',
 'seniorcitizen',
 'partner',
 'dependents',
 'tenure',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'monthlycharges',
 'totalcharges',
 'churn']

In [10]:
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')

print("Missing in totalcharges:", df['totalcharges'].isna().sum())

before = df.shape[0]
df = df.dropna()
after = df.shape[0]

print("Dropped rows:", before - after)
print("Remaining rows:", after)


Missing in totalcharges: 11
Dropped rows: 11
Remaining rows: 7032


In [11]:
df['churn'] = df['churn'].map({'Yes': 1, 'No': 0})

df['churn'].value_counts(normalize=True)


0    0.734215
1    0.265785
Name: churn, dtype: float64

In [12]:
# Features (drop churn + customerid)
X = df.drop(['churn', 'customerid'], axis=1, errors='ignore')
y = df['churn']

X.head()


,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [13]:
# Turn categorical columns into numeric dummies
X = pd.get_dummies(X, drop_first=True)

X.head(), X.shape


(   seniorcitizen  tenure  monthlycharges  totalcharges  gender_Male  \
 0              0       1           29.85         29.85            0   
 1              0      34           56.95       1889.50            1   
 2              0       2           53.85        108.15            1   
 3              0      45           42.30       1840.75            1   
 4              0       2           70.70        151.65            0   
 
    partner_Yes  dependents_Yes  phoneservice_Yes  \
 0            1               0                 0   
 1            0               0                 1   
 2            0               0                 1   
 3            0               0                 0   
 4            0               0                 1   
 
    multiplelines_No phone service  multiplelines_Yes  ...  \
 0                               1                  0  ...   
 1                               0                  0  ...   
 2                               0                  0  ...  

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((5625, 30), (1407, 30))

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [16]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1033
           1       0.65      0.57      0.61       374

    accuracy                           0.80      1407
   macro avg       0.75      0.73      0.74      1407
weighted avg       0.80      0.80      0.80      1407

ROC-AUC: 0.8363483649202004


In [17]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coef': model.coef_[0]
})

feature_importance['abs_coef'] = feature_importance['coef'].abs()

feature_importance.sort_values('abs_coef', ascending=False).head(15)


,feature,coef,abs_coef
25,contract_Two year,-1.313398,1.313398
24,contract_One year,-0.743752,0.743752
10,internetservice_Fiber optic,0.682867,0.682867
7,phoneservice_Yes,-0.509517,0.509517
13,onlinesecurity_Yes,-0.454034,0.454034
19,techsupport_Yes,-0.407631,0.407631
28,paymentmethod_Electronic check,0.388195,0.388195
8,multiplelines_No phone service,0.308298,0.308298
26,paperlessbilling_Yes,0.295868,0.295868
9,multiplelines_Yes,0.274523,0.274523


In [18]:
X_test_copy = X_test.copy()
X_test_copy['churn_actual'] = y_test.values
X_test_copy['churn_probability'] = y_prob

output_path = "data/processed/churn_scored.csv"
X_test_copy.to_csv(output_path, index=False)

output_path


'data/processed/churn_scored.csv'

In [19]:
import joblib

model_path = "models/churn_logistic_model.pkl"
joblib.dump(model, model_path)

print("Model saved to", model_path)


Model saved to models/churn_logistic_model.pkl


In [20]:
loaded_model = joblib.load("models/churn_logistic_model.pkl")

# Quick prediction check
loaded_model.predict(X_test.iloc[:5])


array([0, 1, 0, 0, 0], dtype=int64)

In [21]:
import os

print("data/processed:", os.listdir("data/processed"))
print("models:", os.listdir("models"))
print("notebooks:", os.listdir("notebooks"))


data/processed: ['churn_scored.csv']
models: ['churn_logistic_model.pkl']
notebooks: []
